## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Vaini,-21.20,-175.20,73.40,88,75,10.29,TO,broken clouds
1,1,Hermanus,-34.42,19.23,55.99,89,100,7.00,ZA,light rain
2,2,Tasiilaq,65.61,-37.64,39.20,60,75,21.92,GL,light intensity shower rain
3,3,Kepanjen,-8.13,112.57,72.90,90,92,2.08,ID,moderate rain
4,4,Iracoubo,5.48,-53.20,86.04,63,16,8.63,GF,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
4,4,Iracoubo,5.48,-53.20,86.04,63,16,8.63,GF,light rain
5,5,Cam Ranh,11.92,109.16,84.20,79,20,4.70,VN,few clouds
7,7,Lazaro Cardenas,17.96,-102.20,83.52,71,0,3.60,MX,clear sky
11,11,Alugan,12.22,125.48,78.73,86,99,4.97,PH,overcast clouds
16,16,Carora,10.18,-70.08,89.69,42,84,2.55,VE,broken clouds
17,17,Nizwa,22.93,57.53,88.21,15,0,5.39,OM,clear sky
18,18,Tabas,33.60,56.92,78.80,20,0,2.24,IR,clear sky
31,31,Hithadhoo,-0.60,73.08,83.97,70,72,8.19,MV,broken clouds
33,33,Georgetown,5.41,100.34,77.00,88,20,3.36,MY,few clouds
46,46,Pachino,36.72,15.09,82.40,58,1,4.70,IT,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                216
City                   216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Country                215
Weather Description    216
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
Weather Description     object
dtype: object

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Location Code"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name,Hotel Location Code
4,Iracoubo,GF,86.04,light rain,5.48,-53.20,,
5,Cam Ranh,VN,84.20,few clouds,11.92,109.16,,
7,Lazaro Cardenas,MX,83.52,clear sky,17.96,-102.20,,
11,Alugan,PH,78.73,overcast clouds,12.22,125.48,,
16,Carora,VE,89.69,broken clouds,10.18,-70.08,,
17,Nizwa,OM,88.21,clear sky,22.93,57.53,,
18,Tabas,IR,78.80,clear sky,33.60,56.92,,
31,Hithadhoo,MV,83.97,broken clouds,-0.60,73.08,,
33,Georgetown,MY,77.00,few clouds,5.41,100.34,,
46,Pachino,IT,82.40,clear sky,36.72,15.09,,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df.loc[index, "Hotel Location Code"] = hotels["results"][0]["place_id"]

    except:
        print("Hotel not found... skipping")
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [18]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name,Hotel Location Code
4,Iracoubo,GF,86.04,light rain,5.48,-53.2,Le Gite D Iracoubo,ChIJAcV9k1hDDo0REJBJbbqcHTc
5,Cam Ranh,VN,84.2,few clouds,11.92,109.16,OYO 598 Peony Hotel,ChIJt7i-AQbtcDERUjY4Osz-2Ws
7,Lazaro Cardenas,MX,83.52,clear sky,17.96,-102.2,Collection O Hotel Quinta Antigua,ChIJ0Xy0XG9cMYQRj_ftS-HNei8
11,Alugan,PH,78.73,overcast clouds,12.22,125.48,BEMBEM RAIN FEED RESORT,ChIJcdw6NGllCTMRRQsaatTrsRM
16,Carora,VE,89.69,broken clouds,10.18,-70.08,Hotel Irpinia,ChIJVbPoGGTth44RTY6h4aut3mE


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No Hotel Found"]


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))